# SQL Анализ венчурных фондов  Startup Investments

<img src="https://code.s3.yandex.net/SQL%20for%20data%20and%20analytics/ER/basic_sql_project_ERD.png" alt="ER"/>

## 1. Отобразите все записи из таблицы company по компаниям, которые закрылись.

In [ ]:
SELECT COUNT(status)
FROM company
WHERE status = 'closed';

## 2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [ ]:
SELECT funding_total
FROM company
WHERE id IN (
    SELECT id
    FROM company
    WHERE category_code = 'news'
    )
AND country_code = 'USA'
ORDER BY funding_total DESC;

## 3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2011 AND 2013
AND term_code = 'cash';

## 4. Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
        last_name,
        twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

## 5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';

## 6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
        SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

## 7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
        MIN(raised_amount),
        MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 AND MIN(raised_amount) <> MAX(raised_amount);

## 8. Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity,
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity,
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity,  
  
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
        CASE
        WHEN invested_companies < 20 THEN 'low_activity'
        WHEN invested_companies >= 100 THEN 'high_activity'
        ELSE 'middle_activity'
        END
FROM fund;

## 9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));

## 10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.  
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.  
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
        MIN(invested_companies),
        MAX(invested_companies),
        AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;

## 11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
         p.last_name,
         e.instituition
FROM people AS p
LEFT OUTER JOIN education AS e ON p.id=e.person_id;

## 12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
WITH
e AS (
    SELECT person_id,
        instituition,
        graduated_at
    FROM education
),
c AS (
    SELECT id,
        name
    FROM company
),
p AS (
SELECT id,
        company_id
FROM people
)    

SELECT c.name,
        COUNT(DISTINCT tab_1.instituition)
FROM c
RIGHT OUTER JOIN  (
SELECT p.company_id,
        e.instituition
FROM p
INNER JOIN e ON p.id=e.person_id
WHERE e.instituition IS NOT NULL) AS tab_1 ON tab_1.company_id=c.id
WHERE c.name IS NOT NULL
GROUP BY c.name
ORDER BY COUNT(DISTINCT tab_1.instituition) DESC
LIMIT 5;

## 13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT name
FROM company
WHERE id IN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
    )
AND status = 'closed';

## 14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT id
FROM people
WHERE company_id IN(
    SELECT id
    FROM company
    WHERE id IN (
        SELECT company_id
        FROM funding_round
        WHERE is_first_round = 1 AND is_last_round = 1
        )
    AND status = 'closed');

## 15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT person_id,
        instituition
FROM education
WHERE person_id IN(
SELECT id
FROM people
WHERE company_id IN(
    SELECT id
    FROM company
    WHERE id IN (
        SELECT company_id
        FROM funding_round
        WHERE is_first_round = 1 AND is_last_round = 1
        )
    AND status = 'closed'));

## 16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT  person_id,
       COUNT(instituition)
FROM education
WHERE person_id IN(
SELECT id
FROM people
WHERE company_id IN(
    SELECT id
    FROM company
    WHERE id IN (
        SELECT company_id
        FROM funding_round
        WHERE is_first_round = 1 AND is_last_round = 1
        )
    AND status = 'closed'))
GROUP BY person_id;

## 17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(co_in)
FROM(
SELECT  person_id,
       COUNT(instituition) AS co_in
FROM education
WHERE person_id IN(
SELECT id
FROM people
WHERE company_id IN(
    SELECT id
    FROM company
    WHERE id IN (
        SELECT company_id
        FROM funding_round
        WHERE is_first_round = 1 AND is_last_round = 1
        )
    AND status = 'closed'))
GROUP BY person_id) AS tab_1;

## 18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [ ]:
SELECT AVG(tab_1.co_in)
FROM (
    SELECT person_id,
            COUNT(instituition) AS co_in
    FROM education
    WHERE person_id IN (
    SELECT id
        FROM people
        WHERE company_id =
            (SELECT id
            FROM company
            WHERE name = 'Facebook'))
    GROUP BY person_id) AS tab_1;

## 19. Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.  
  
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
        c.name AS name_of_company,
        id_list.raised_amount AS amount
FROM (SELECT company_id,
        fund_id,
        raised_amount
FROM investment AS inv
RIGHT OUTER JOIN (

SELECT id,
        raised_amount
FROM funding_round
WHERE EXTRACT(YEAR FROM CAST(funded_at AS timestamp)) BETWEEN 2012 AND 2013
AND company_id IN (
    SELECT id
    FROM company
    WHERE milestones > 6
)) AS inv_list ON inv_list.id=inv.funding_round_id) AS id_list
INNER JOIN company AS c ON c.id=id_list.company_id
INNER JOIN fund AS f ON f.id=id_list.fund_i;

## 20. Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.  
  
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.  
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT com.name AS acquiring_company,
        
        tab_1.price_amount,
        tab_1.acquired_company,
        tab_1.funding_total,
        ROUND(tab_1.price_amount / tab_1.funding_total)
        
        
FROM 
(    SELECT acquiring_company_id,
         price_amount,
         c.name AS acquired_company,
            funding_total
    FROM (
        SELECT acquiring_company_id,
                acquired_company_id,
                price_amount
        FROM acquisition
        WHERE price_amount > 0) AS acq
    LEFT OUTER JOIN company AS c ON acq.acquired_company_id=c.id
    WHERE funding_total > 0) AS tab_1
LEFT OUTER JOIN company AS com ON tab_1.acquiring_company_id=com.id
ORDER BY tab_1.price_amount DESC, ROUND(tab_1.price_amount / tab_1.funding_total) DESC
LIMIT 10;

## 21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT name,
        ci.i_month
FROM company AS c
RIGHT OUTER JOIN 
    (SELECT company_id,
            EXTRACT(MONTH FROM CAST(funded_at AS timestamp)) AS i_month
    FROM funding_round
    WHERE EXTRACT(YEAR FROM CAST(funded_at AS timestamp)) BETWEEN 2010 AND 2013
    AND raised_amount > 0) AS ci ON c.id=ci.company_id
WHERE category_code = 'social';

## 22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
a AS (
SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)) AS inv_month,
        inv.fund_id
    FROM funding_round AS fr
    LEFT OUTER JOIN investment AS inv ON fr.id=inv.funding_round_id
    WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN 2010 AND 2013
    AND inv.fund_id IN(
        SELECT id
    FROM fund
    WHERE country_code = 'USA')
),

b AS (
    SELECT COUNT(DISTINCT f.name) AS fund_count,
            a.inv_month
    FROM a
    LEFT OUTER JOIN fund AS f ON a.fund_id=f.id
    GROUP BY a.inv_month
),

r AS (
    SELECT EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS acq_month,
            COUNT(acquired_company_id) AS count_com,
            SUM(price_amount) AS sum_amont
    FROM acquisition
    WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN 2010 AND 2013
    GROUP BY EXTRACT(MONTH FROM CAST(acquired_at AS timestamp))
    )

SELECT acq_month,
        count_com,
        sum_amont,
        fund_count
FROM b
INNER JOIN r ON b.inv_month=r.acq_month;

## 23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH

inv_2011 AS (
            SELECT AVG(funding_total) AS total_2011,
                    country_code
            FROM company
            GROUP BY country_code, EXTRACT(YEAR FROM founded_at)
            HAVING EXTRACT(YEAR FROM founded_at) = 2011
            ),
               
inv_2012 AS (
            SELECT AVG(funding_total) AS total_2012,
                      country_code
            FROM company
            GROUP BY country_code, EXTRACT(YEAR FROM founded_at)
            HAVING EXTRACT(YEAR FROM founded_at) = 2012
            ),

inv_2013 AS (
            SELECT AVG(funding_total) AS total_2013,
                    country_code
            FROM company
            GROUP BY country_code, EXTRACT(YEAR FROM founded_at)
            HAVING EXTRACT(YEAR FROM founded_at) = 2013
            )

SELECT inv_2011.country_code,
       inv_2011.total_2011,
       inv_2012.total_2012,
       inv_2013.total_2013
FROM inv_2011 
INNER JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013 ON inv_2011.country_code = inv_2013.country_code
ORDER BY inv_2011.total_2011 DESC;